In [ ]:
import pandas as pd
df = pd.read_csv("data_evaluation.csv")
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
model = api.load("word2vec-google-news-300")
lemmatizer = nltk.WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
df.head()

,A,B,C,D,E,F,G,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"App is useful to certain phone brand ,,,,it is...","Worst customer service. Very scripted, refuse ...",For an app who's sole purpose is to stream ent...,Have had netflix since it first went digital. ...,I have the premium account and the past few mo...,Expensive and poor picture quality,"dissapointment, netflix has took many good gam...",NaN,NaN,NaN
1,"I just installed the app. But when I open it, ...","After an update, playback specification will a...",The app is not working. The whole screen is bl...,THIS APP GETS RID OF THE BEST SHOWS! LIKE JULI...,I don't understand why I have to register with...,I am going to unlike all and any documentaries...,"plays well. navigation, looking for films, is ...",NaN,NaN,NaN
2,I enjoy Netflix as much as anyone else but why...,I just wish all foreign TV shows (K- drama) Au...,I'm not even able to download any movies or se...,I am a truck driver so my location changes all...,Honestly the app works just fine and I general...,"there isn't inside out, Heather the musical, T...",Good but fricking expensive. Family can't shar...,NaN,NaN,NaN
3,"Overall Satisfactory, one suggestion might be ...",Works properly but sometimes it lags and not m...,Fast/complete,"I enjoy the app fairly well, but i can not sta...",Just a small feature request - along with disp...,Netflix is always the best. But I wish it woul...,My experience is very good but most of the dra...,NaN,NaN,NaN
4,Best app fOr movies,I love to watch the movie's you have on here a...,"Great app with good variety, but currently you...",Netflix is ​​a great medium for movies and TV ...,Because you can watch movies and relax while i...,maybe i share my experience to how use the net...,I love cause it has what i want go Netflix but...,NaN,NaN,NaN


In [6]:


df.values[1:, 3:] = df.values[0, 3:]


def get_scores(f, alpha = 1, beta = 1):
  ctr = 0
  for i in df.values:
    ctr+=1
    anc = i[0]
    pos = list(i[[1, 2]])
    neg = list(i[3:])
    anc = f(anc)
    pos = list(map(f, pos))
    neg = list(map(f, neg))
    anc_dot = lambda x: np.array(anc).T @ np.array(x)
    pos = list(map(anc_dot, pos))
    neg = list(map(anc_dot, neg))
    print("data", ctr, "score = ", alpha*sum(pos)/2 - beta*sum(neg)/10)


import string
def preprocess(x):
  x = str(x)
  x = x.lower()
  x = "".join([char for char in x if char not in string.punctuation])
  words = word_tokenize(x)
  words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  x = ' '.join(words)
  return x


import numpy as np
def word2vec_f(x):
  x = preprocess(x)
  x = x.split(' ')
  embs = [] # embeddings = sentence*300
  for i in x:
    try:
      embs.append(model[i])
    except:
      pass
  x = np.array(embs)
  x = np.mean(x, axis = 0)
  if(embs == []):
    x = np.zeros((300, ))
  # print(x.shape)
  return x



In [7]:
get_scores(word2vec_f, 1, 1)

data 1 score =  0.391947528719902
data 2 score =  0.49126717746257786
data 3 score =  0.3150947064161301
data 4 score =  -0.11552698612213136
data 5 score =  0.6391087174415588
data 6 score =  3.2024851799011227
data 7 score =  3.2024851799011227
data 8 score =  3.2024851799011227
data 9 score =  3.2024851799011227
data 10 score =  3.2024851799011227


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
!pip install transformers

In [11]:
import numpy as np
import pandas as pd
import torch as nn
import transformers
import json
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [12]:

def bert_f(x):
  x = preprocess(x)
  inp = nn.tensor(tokenizer.encode(x))[:512]
  inp = inp.unsqueeze(0)
  outputs = model(inp)
  return np.array(outputs[0].detach()[0][0])

In [13]:
get_scores(bert_f, 1, 1)

data 1 score =  55.77423248291015
data 2 score =  53.81710357666016
data 3 score =  56.1828125
data 4 score =  53.81981201171875
data 5 score =  54.853187561035156
data 6 score =  62.299530029296875
data 7 score =  62.299530029296875
data 8 score =  62.299530029296875
data 9 score =  62.299530029296875
data 10 score =  62.299530029296875


In [14]:
model_glove = api.load("glove-twitter-200")


[==================================================] 100.0% 758.5/758.5MB downloaded


In [18]:
def glove_f(x):
  x = preprocess(x)
  x = x.split(' ')
  embs = []
  for i in x:
    try:
      embs.append(model_glove[i])
    except:
      pass
  x = np.array(embs)
  x = np.mean(x, axis = 0)
  if(embs == []):
    x = np.zeros((300, ))
  # print(x.shape)
  return x

In [19]:
get_scores(glove_f, 1, 1)

data 1 score =  6.0080766677856445
data 2 score =  8.035081386566162
data 3 score =  7.562138080596924


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 300 is different from 200)